### Read meteorological data
Folder with [original data](https://drive.google.com/drive/folders/1me2IpDY3om6IRKv_WMT5W6Qw0r9DI74C)

In [1]:
import os
import zipfile
import pandas as pd
import glob
import keplergl
import geopandas as gpd
import matplotlib.pyplot as plt
import csv
from datetime import datetime
import re
import numpy as np
from pyproj import Proj
%run Meteo_utils.ipynb

In [2]:
os.listdir('data')

['.DS_Store',
 'Estaciones_meteorologicas.csv',
 'Inventario Estaciones Meteo.xls',
 '.ipynb_checkpoints',
 'PROCESSED',
 'RAW']

## Automate data extraction from Drive downloads

In [40]:
path = '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW'
ziplist = glob.glob(f'{path}/*.zip')
#ziplist = glob.glob(f'data/*.zip')

In [41]:
ziplist

['/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/14-HGPT-MT-09.- Estacion Calamaca Convenio-20210716T093623Z-001.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/25-INAMHI-M0126.- Patate-20210716T093732Z-001.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/23-INAMHI-M0128.- Colegio P. F. Cevallos-20210716T093721Z-001.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/15-HGPT-MT-12.- Estacion Pampas de Salasaca-20210716T093632Z-001.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/17-HGPT-MT-13.- Estacion Tasinteo-20210716T093648Z-001.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/17-HGPT-MT-13.- Estacion Tasinteo-20210716T093648Z-002.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/09-HGPT-MT-02.- Estacion Colegio A. J. Sucre-20210625T055503Z-001.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/27-INAMHI-M1243.- Cunchibamba-20210716T093750Z-001.zip',
 '/Users/tamarah

In [5]:
folder = 0

In [9]:
zf = zipfile.ZipFile(f'{ziplist[folder]}')
zf.namelist()[0:10]

['14-HGPT-MT-09.- Estacion Calamaca Convenio/2014-07-25/Calamaca_5min_20140804_subir.csv',
 '14-HGPT-MT-09.- Estacion Calamaca Convenio/2013-05-07/Calamaca Convenio_1hora_20130515.xls',
 '14-HGPT-MT-09.- Estacion Calamaca Convenio/2014-07-25/Calamaca_5min_20140804.csv',
 '14-HGPT-MT-09.- Estacion Calamaca Convenio/2016-04-21/8310_5min_20161024_subir.csv',
 '14-HGPT-MT-09.- Estacion Calamaca Convenio/2014-11-17/Calamaca_5min_20141117_subir.csv',
 '14-HGPT-MT-09.- Estacion Calamaca Convenio/Calamaca Convenio Todo.csv',
 '14-HGPT-MT-09.- Estacion Calamaca Convenio/2013-05-07/Calamaca Convenio_1hora_20130515.csv',
 '14-HGPT-MT-09.- Estacion Calamaca Convenio/2015-07-03/8310_5min_20150715_subir.csv',
 '14-HGPT-MT-09.- Estacion Calamaca Convenio/2016-01-21/8310_5min_20160125_subir.csv',
 '14-HGPT-MT-09.- Estacion Calamaca Convenio/2013-08-15/Calamaca_5min_20130821.csv']

In [10]:
### exclude macos files (only if copying data if using Drive directly these don't appear)
r = re.compile('^__MACOSX') # only csv files
exclude = list(filter(r.match, zf.namelist()[0:10])) 
clean = list(set(zf.namelist()[0:10])-set(exclude))

In [12]:
files_by_date = order_meteo_zip(path, folder =folder)

In [309]:
files_by_date

['27-INAMHI-M1243.- Cunchibamba/._Ordenado.csv',
 '27-INAMHI-M1243.- Cunchibamba/Ordenado.csv']

In [14]:
df = read_meteo_csv(path =path,folder = folder,file = files_by_date[0])

Success sep ";", latin encoding
file =14-HGPT-MT-09.- Estacion Calamaca Convenio/2013-05-07/Calamaca Convenio_1hora_20130515.csv, cols = 7


In [15]:
df.head()

,Date,Time,Variable,Value,Type,Freq,file_name
1,03/03/2013,02:00:00,WindMnDirUnit,171.73,G,1hora,14-HGPT-MT-09.- Estacion Calamaca Convenio/201...
2,03/03/2013,02:00:00,WindMinSpdSclr,0.21,G,1hora,14-HGPT-MT-09.- Estacion Calamaca Convenio/201...
3,03/03/2013,02:00:00,WindMaxSpdSclr,2.94,G,1hora,14-HGPT-MT-09.- Estacion Calamaca Convenio/201...
4,03/03/2013,02:00:00,WindMaxDir,174.47,G,1hora,14-HGPT-MT-09.- Estacion Calamaca Convenio/201...
5,03/03/2013,03:00:00,BAT,12.91,G,1hora,14-HGPT-MT-09.- Estacion Calamaca Convenio/201...


In [16]:
files_by_date[0].split('/')[-1].replace(" ","")

'CalamacaConvenio_1hora_20130515.csv'

In [105]:
station = files_by_date[0].split('/')[0].replace(" ","").replace(".","")
all_var = get_unique_variables(files_by_date,path =path,folder = folder)

Success sep ";", latin encoding
file =24-INAMHI-M0380.- HUambalo/2015/julio_diciembre.csv, cols = 4
Success sep ";", latin encoding
file =24-INAMHI-M0380.- HUambalo/2016/Enero_Abril.csv, cols = 4
Success sep ";", latin encoding
file =24-INAMHI-M0380.- HUambalo/Mayo-Octubre_subir.csv, cols = 4
Success sep ";", latin encoding
file =24-INAMHI-M0380.- HUambalo/2015/Julio- Octubre_subir.csv, cols = 4
Success sep ";", latin encoding
file =24-INAMHI-M0380.- HUambalo/2013/2013_subir.csv, cols = 4
Success sep ";", latin encoding
file =24-INAMHI-M0380.- HUambalo/2014/2014_huambalo_subir.csv, cols = 4
Success sep ";", latin encoding
file =24-INAMHI-M0380.- HUambalo/2015/Enero_Julio.csv, cols = 4


In [17]:
## Variable names:
# BAT  is  battery
# Not very clear: 'H_M Min','WindMnSpdSclr','WindMnSpdS','WindMnDirUnit'

replace_values ={
# Temperature
    'ATAvg' : ['AT1HrAvg', 'AT5minAvg', 'ATAvg','Temperatura'],
    'ATMin' : ['AT1HrMin', 'AT5minMin','T_Min', 'T Min'],
    'ATMax' : ['AT1HrMax','AT5minMax', 'T_Max', 'T Max'],
 
 # Relative Humidity
    'RHAvg' : ['Humedad','RH5minAvg','RHAvg','RelHumidAvg', 'Humedad R', 'RH5m2015'],
    'RHMin' : ['H_Min','RH5minMin','RelHumidMin','H_M Min'],
    'RHMax' : ['H_Max', 'RH5minMax', 'RelHumidMax','H_R Max'],

 # Wind Speed
    'WSAvg' : ['Velocidad','WS5minAvg','WSAvg','WindMnSpdSclr','WindMnSpdS'],
    'WSMin' : ['WindMinSpdSclr','WS5minMin'],
    'WSMax' : ['WindMaxSpdSclr','WS5minMax'],

 # Wind Direction
    'WDAvg' : ['Direccion', 'WDAvg','WD5minAvg','WindMnDirUnit','Dirección'],
    'WDMin' : ['WD5minMin'],
    'WDMax' : ['WD5minMax','WindMaxDir'],

 # Rain
    'PAvg' : ['Precipitacion','TB1hrAcc','TB1minAcc','TB5minAcc','Precipitación']
}

var_list = ['ATAvg','ATMin','ATMax','RHAvg','RHMin','RHMax', 'WSAvg','WSMin','WSMax','WDAvg','WDMin', 'WDMax','PAvg']
#sorted(var_list)

In [76]:
df = read_meteo_csv(path =path,folder = 0,file = files_by_date[0])
df2 = reformat_df(df=df, replace_values=replace_values)
df2.dtypes

Success sep ";", latin encoding
file =24-INAMHI-M0380.- HUambalo/2015/julio_diciembre.csv, cols = 4


Date_Time    datetime64[ns]
Type                float64
Freq                 object
file_name           float64
ATAvg               float64
ATMax               float64
ATMin               float64
BAT                 float64
PAvg                float64
RHAvg               float64
RHMax               float64
RHMin               float64
TB1hrAcc            float64
TB1minAcc           float64
TB5minAcc           float64
WDAvg               float64
WDMax               float64
WDMin               float64
WSAvg               float64
WSMax               float64
WSMin               float64
st_name             float64
dtype: object

In [77]:
len(files_by_date)

7

## Process files in loop

In [18]:
%run Meteo_utils.ipynb

In [322]:
## Variable names:
# BAT  is  battery
# Not very clear: 'H_M Min','WindMnSpdSclr','WindMnSpdS','WindMnDirUnit'
replace_values ={
# Temperature
    'ATAvg' :['AT1HrAvg', 'AT5minAvg', 'ATAvg','Temperatura','Temperatuta','TEMPERATURA AIRE Â°C PROM'],
    'ATMin' : ['AT1HrMin', 'AT5minMin','T_Min', 'T Min','TEMPERATURA AIRE Â°C MIN'],
    'ATMax' : ['AT1HrMax','AT5minMax', 'T_Max', 'T Max','TEMPERATURA AIRE Â°C MAX'],
 
 # Relative Humidity
    'RHAvg' : ['Humedad','RH5minAvg','RHAvg','RelHumidAvg', 'Humedad R', 'RH5m2015','HUMEDAD RELATIVA DEL AIRE % PROM'],
    'RHMin' : ['H_Min','RH5minMin','RelHumidMin','H_M Min','HUMEDAD RELATIVA DEL AIRE % MIN'],
    'RHMax' : ['H_Max', 'RH5minMax', 'RelHumidMax','H_R Max','HUMEDAD RELATIVA DEL AIRE % MAX'],

 # Wind Speed
    'WSAvg' : ['Velocidad','WS5minAvg','WSAvg','WindMnSpdSclr','WindMnSpdS','VIENTO VELOCIDAD m/s INST'],
    'WSMin' : ['WindMinSpdSclr','WS5minMin'],
    'WSMax' : ['WindMaxSpdSclr','WS5minMax'],

 # Wind Direction
    'WDAvg' : ['Direccion', 'WDAvg','WD5minAvg','WindMnDirUnit','Dirección','VIENTO DIRECCION Â° INST'],
    'WDMin' : ['WD5minMin'],
    'WDMax' : ['WD5minMax','WindMaxDir'],

 # Rain
    'PAvg' : ['Precipitacion','TB1hrAcc','TB1minAcc','TB5minAcc','Precipitación','PRECIPITACION mm SUM']
}

var_list = ['ATAvg','ATMin','ATMax','RHAvg','RHMin','RHMax', 'WSAvg','WSMin','WSMax','WDAvg','WDMin', 'WDMax','PAvg']
#sorted(var_list)

In [310]:
path = '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW'
ziplist = glob.glob(f'{path}/*.zip')
ziplist

['/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/14-HGPT-MT-09.- Estacion Calamaca Convenio-20210716T093623Z-001.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/25-INAMHI-M0126.- Patate-20210716T093732Z-001.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/23-INAMHI-M0128.- Colegio P. F. Cevallos-20210716T093721Z-001.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/15-HGPT-MT-12.- Estacion Pampas de Salasaca-20210716T093632Z-001.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/17-HGPT-MT-13.- Estacion Tasinteo-20210716T093648Z-001.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/17-HGPT-MT-13.- Estacion Tasinteo-20210716T093648Z-002.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/09-HGPT-MT-02.- Estacion Colegio A. J. Sucre-20210625T055503Z-001.zip',
 '/Users/tamarahuete/Documents/Github_repos/TFM21/data/RAW/27-INAMHI-M1243.- Cunchibamba-20210716T093750Z-001.zip',
 '/Users/tamarah

In [165]:
file, folder

(1, 4)

In [389]:
path = '/Users/tamarahuete/Documents/Github_repos/TFM21/data'
folder = 9
files_by_date = order_meteo_zip(path=f'{path}/RAW/', folder =folder)
#get_unique_variables(files_by_date,path =f'{path}/RAW/',folder = folder)

for file in range(50, len(files_by_date)):
        print(f'{file}/{len(files_by_date)}')
        df = read_meteo_csv(path =f'{path}/RAW',folder = folder,file = files_by_date[file])    
        df2 = reformat_df(df=df, replace_values=replace_values)
        master_df = master_df.append(df2)
        
export_name = f'{files_by_date[0].split("/")[0].replace(" ","").replace(".","")}'
repeat = list(set([export_name +'.csv']) & set(os.listdir(f'{path}/PROCESSED')))
if len(repeat)>0:
    master_df.to_csv(f'{path}/PROCESSED/{export_name}_{len(repeat)+1}.csv')
else :
    master_df.to_csv(f'{path}/PROCESSED/{export_name}.csv')

50/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2018-06-19/MULA CORRAL_5MIN_20180620.csv, cols = 7
51/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2018-06-19/MULA CORRAL_5MIN_20180620 - copia.csv, cols = 7
52/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2018-08-22/MULA CORRAL_5MIN_20180822.csv, cols = 7
53/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2018-08-22/MULA CORRAL_1MIN_20180823.csv, cols = 7
54/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2018-09-27/MULA CORRAL_5MIN_201809270.csv, cols = 7
55/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2018-09-27/MULA CORRAL_1MIN_201809270.csv, cols = 7
56/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2018-12-27/MULA CORRAL_5MIN_20181227.csv, cols = 7
57/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estaci

In [313]:
df

,PAvg,Freq,file_name,Date_Time,Type,WSMin,RHMin,WDMax,WSMax,ATAvg,RHMax,ATMax,ATMin,WDAvg,WDMin,WSAvg,RHAvg
0,"5,10",1day,27-INAMHI-M1243.- Cunchibamba/Ordenado.csv,2013-01-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"5,60",1day,27-INAMHI-M1243.- Cunchibamba/Ordenado.csv,2013-02-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"19,20",1day,27-INAMHI-M1243.- Cunchibamba/Ordenado.csv,2013-03-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"0,00",1day,27-INAMHI-M1243.- Cunchibamba/Ordenado.csv,2013-04-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"5,50",1day,27-INAMHI-M1243.- Cunchibamba/Ordenado.csv,2013-05-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,"1,00",1day,27-INAMHI-M1243.- Cunchibamba/Ordenado.csv,2013-12-27 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,"2,20",1day,27-INAMHI-M1243.- Cunchibamba/Ordenado.csv,2013-12-28 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,"1,20",1day,27-INAMHI-M1243.- Cunchibamba/Ordenado.csv,2013-12-29 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,"0,00",1day,27-INAMHI-M1243.- Cunchibamba/Ordenado.csv,2013-12-30 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [380]:
files_by_date = order_meteo_zip(path=f'{path}/RAW/', folder =8,only_csv=False)
files_by_date

['20-Querochaca/2016/M0258 QUEROCHACA 2016.csv']

In [381]:
path = '/Users/tamarahuete/Documents/Github_repos/TFM21/data'
ziplist = glob.glob(f'{path}/RAW/*.zip')
for folder in range(9, len(ziplist)):
    
    files_by_date = order_meteo_zip(path=f'{path}/RAW/', folder =folder)
    #get_unique_variables(files_by_date,path =f'{path}/RAW/',folder = folder)

    master_df =pd.DataFrame()
    for file in range(0, len(files_by_date)):
        print(f'{file}/{len(files_by_date)}')
        df = read_meteo_csv(path =f'{path}/RAW',folder = folder,file = files_by_date[file])    
        df2 = reformat_df(df=df, replace_values=replace_values)
        master_df = master_df.append(df2)
        
    export_name = f'{files_by_date[0].split("/")[0].replace(" ","").replace(".","")}'
    repeat = list(set([export_name +'.csv']) & set(os.listdir(f'{path}/PROCESSED')))
    if len(repeat)>0:
        master_df.to_csv(f'{path}/PROCESSED/{export_name}_{len(repeat)+1}.csv')
    else :
        master_df.to_csv(f'{path}/PROCESSED/{export_name}.csv')

0/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2013-03-19/Mula Corral_1hora_20130326.csv, cols = 7
1/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2013-05-07/Mula Corral_1hora_20130515.csv, cols = 7
2/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2013-06-13/Mula Corral_5min_20130625.csv, cols = 7
3/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2013-10-03/Mula Corral_5min_20131007.csv, cols = 7
4/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2014-01-04/Mula Corral_5min_20140116_subir.csv, cols = 8
5/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2014-01-04/Mula Corral_5min_20140116.csv, cols = 7
6/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2014-04-08/Mula Corral_5min_20140422_subir.csv, cols = 8
7/60
Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion M

KeyError: 'Date'

In [374]:
%run Meteo_utils.ipynb

In [382]:
file, folder

(47, 9)

In [383]:
df = read_meteo_csv(path =f'{path}/RAW',folder = folder,file = files_by_date[file])    
#df2 = reformat_df(df=df, replace_values=replace_values)

Success sep ";", latin encoding
file =16-HGPT-MT-10.- Estacion Mula Corral/2018-06-19/dia_prueba_2.csv, cols = 9


In [384]:
df

,"10/21/2016,11:49:00",",",TB1minAcc,",.1",0.00,",.2",",.3",G,file_name
0,"10/21/2016,11:49:00",",",RHAvg,",",101.12,",",",",G,16-HGPT-MT-10.- Estacion Mula Corral/2018-06-1...
1,"10/21/2016,11:49:00",",",ATAvg,",",-4.00,",",",",G,16-HGPT-MT-10.- Estacion Mula Corral/2018-06-1...
2,"10/21/2016,11:49:00",",",WDAvg,",",238.00,",",",",G,16-HGPT-MT-10.- Estacion Mula Corral/2018-06-1...
3,"10/21/2016,11:49:00",",",WSAvg,",",2.20,",",",",G,16-HGPT-MT-10.- Estacion Mula Corral/2018-06-1...
4,"10/21/2016,11:50:00",",",TB1minAcc,",",0.00,",",",",G,16-HGPT-MT-10.- Estacion Mula Corral/2018-06-1...
...,...,...,...,...,...,...,...,...,...
3649,"10/21/2016,23:59:00",",",TB1minAcc,",",0.00,",",",",G,16-HGPT-MT-10.- Estacion Mula Corral/2018-06-1...
3650,"10/21/2016,23:59:00",",",RHAvg,",",9.89,",",",",G,16-HGPT-MT-10.- Estacion Mula Corral/2018-06-1...
3651,"10/21/2016,23:59:00",",",ATAvg,",",64.39,",",",",G,16-HGPT-MT-10.- Estacion Mula Corral/2018-06-1...
3652,"10/21/2016,23:59:00",",",WDAvg,",",188.00,",",",",G,16-HGPT-MT-10.- Estacion Mula Corral/2018-06-1...


In [348]:
df2 = reformat_df(df=df, replace_values=replace_values)

In [375]:
folder =8

In [376]:
ziplist = glob.glob(f'{path}/RAW/*.zip')
zf = zipfile.ZipFile(f'{ziplist[folder]}')

In [299]:
if len(df.columns) == 1:
        df = df[df.columns[0]].str.split(',',expand=True)
        if len(df.columns) == 1:
            df = df[df.columns[0]].str.split('.',expand=True)
            df[3] =df[3].astype(str).str.cat(df[4].astype(str),sep=".")
            df.drop(columns=4, inplace=True)
        df = df.iloc[1:]   

In [303]:
df

,Date,Time,Variable,Value,Type,Freq,file_name
1,06/21/2013,10:45:00,TB5minAcc,0.00,G,5min,09-HGPT-MT-02.- Estacion Colegio A. J. Sucre/2...
2,06/21/2013,10:45:00,AT5minAvg,14.55,B,5min,09-HGPT-MT-02.- Estacion Colegio A. J. Sucre/2...
3,06/21/2013,10:45:00,AT5minMin,14.41,G,5min,09-HGPT-MT-02.- Estacion Colegio A. J. Sucre/2...
4,06/21/2013,10:45:00,AT5minMax,14.69,G,5min,09-HGPT-MT-02.- Estacion Colegio A. J. Sucre/2...
5,06/21/2013,10:45:00,RH5minAvg,71.89,B,5min,09-HGPT-MT-02.- Estacion Colegio A. J. Sucre/2...
...,...,...,...,...,...,...,...
153306,08/01/2013,09:25:00,WS5minAvg,1.10,G,5min,09-HGPT-MT-02.- Estacion Colegio A. J. Sucre/2...
153307,08/01/2013,09:25:00,WS5minMin,0.95,G,5min,09-HGPT-MT-02.- Estacion Colegio A. J. Sucre/2...
153308,08/01/2013,09:25:00,WS5minMax,1.30,G,5min,09-HGPT-MT-02.- Estacion Colegio A. J. Sucre/2...
153309,08/01/2013,09:25:00,WD5minMax,-65.03,G,5min,09-HGPT-MT-02.- Estacion Colegio A. J. Sucre/2...
